In [1]:
'''
File name : OpenRetina_manipulations.ipynb
Authors : Pierre Albiges, Victor Boutin and Laurent Perrinet
Date created : 2017-04-25
Date last modified : 2017-04-25
Object : Apprehend the OpenRetina project through the manipulation of the Photoreceptor class

See https://github.com/laurentperrinet/openRetina/blob/master/src/openRetina.py
'''

class PhotoReceptor :
    def __init__(self, w, h, cam_id = 0, DOWNSCALE = 0, verbose = True ) :
        self.sleep_time = 2 #Letting the camera warming up for 2 seconds
        print 'h = ', h, 'w = ', w